censusdata docs: https://jtleider.github.io/censusdata/example1.html</br>
for la county code: https://notary.cdn.sos.ca.gov/forms/notary-county-codes.pdf</br>
example of censustract package: https://towardsdatascience.com/mapping-us-census-data-with-python-607df3de4b9c


In [99]:
import pandas as pd
import censusdata

In [105]:
def search_census_concepts(search_term):
    sample = censusdata.search('acs5', 2019,'concept', search_term)
    return sample

In [114]:
search_census_concepts('family income')

[('B10010_001E',
  'MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN',
  'Estimate!!Median family income in the past 12 months--!!Total:'),
 ('B10010_002E',
  'MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN',
  'Estimate!!Median family income in the past 12 months--!!Grandparent householder and/or spouse responsible for grandchildren under 18 years (dollars)'),
 ('B10010_003E',
  'MEDIAN FAMILY INCOME FOR FAMILIES WITH GRANDPARENT HOUSEHOLDERS AND/OR SPOUSES LIVING WITH OWN GRANDCHILDREN UNDER 18 YEARS BY RESPONSIBILITY FOR OWN GRANDCHILDREN AND PRESENCE OF PARENT OF GRANDCHILDREN',
  'Estimate!!Median family income in the past 12 months--!!Grandparent householder and/or spouse r

In [121]:
tract_pops = censusdata.download('acs5', 
             2015,
             censusdata.censusgeo([('state', '06'), ('county', '037'), ('tract','*')]),
             ['B01003_001E','B25001_001E','B19113_001E']).reset_index(drop=False)\
             .rename(columns={'index':'census tract',
                              'B01003_001E':'tract_pop',
                              'B25001_001E':'housing_units',
                              'B19113_001E':'med_income_12mo_adj'})
tract_pops['census tract'] = [str(i)[13:-88] for i in tract_pops['census tract']]
tract_pops

,census tract,tract_pop,housing_units,med_income_12mo_adj
0,5301.02,5308,1816,47392.0
1,1046.10,3385,713,51042.0
2,1065.10,5151,1643,85344.0
3,1066.41,2145,799,109583.0
4,1132.13,4714,1680,90230.0
...,...,...,...,...
2341,5035.01,6711,2083,61994.0
2342,5035.02,4154,1292,72690.0
2343,5041.02,0,0,-666666666.0
2344,2657,5184,2695,136161.0


In [ ]:
tract_pops.med_inc

In [76]:
tract_pops.tract_pop.describe()

count     2346.000000
mean      4278.937766
std       1556.788900
min          0.000000
25%       3238.000000
50%       4175.500000
75%       5279.000000
max      12916.000000
Name: tract_pop, dtype: float64

In [96]:
low_high = [(4278 - 1556),(4278 + 1556)]

tract_pops.loc[tract_pops['tract_pop']<low_high[0],'level']='low'
tract_pops.loc[tract_pops['tract_pop']>low_high[1],'level']='high'
tract_pops['level'] = tract_pops['level'].fillna('mid')

tract_pops

,census tract,tract_pop,level
0,5301.02,5308,mid
1,1046.10,3385,mid
2,1065.10,5151,mid
3,1066.41,2145,low
4,1132.13,4714,mid
...,...,...,...
2341,5035.01,6711,high
2342,5035.02,4154,mid
2343,5041.02,0,low
2344,2657,5184,mid
